In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
!pwd

/home/sid/ds_nudge_up/playground


In [3]:
import matplotlib.pyplot as plt

In [4]:
import sys
sys.path += ["../"]

In [5]:
import os
import src.utils as utl
rasterio_config = utl.load_yaml_config("rasterioc_config.yaml")
os.environ.update(rasterio_config)

In [6]:
from src.featurize import *

In [52]:
featurization_params = utl.load_yaml_config("featurisation.yaml")
satellite_config = utl.load_yaml_config("satellite_config.yaml")
satellite_config = satellite_config[featurization_params['satellite_image_params']['satellite_name']]
data_sources = utl.load_yaml_config('data_catalog.yaml')

In [53]:
points_gdf = utl.load_points_gdf(**data_sources['request_points'])

In [57]:
%%time
points_gdf_with_stac = fetch_image_refs(points_gdf.loc[:20, :], featurization_params['dask']['n_partitions'], 
                                        featurization_params['satellite_image_params'])

/home/sid/miniconda3/envs/nudge_up/lib/python3.10/site-packages/geopandas/geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


CPU times: user 753 ms, sys: 27.8 ms, total: 781 ms
Wall time: 1.89 s


In [58]:
mini_stac_gdf = points_gdf_with_stac[:10]

In [82]:
mini_stac_gdf.stac_item.iloc[3]

ID: S2B_MSIL2A_20220326T054639_R048_T43SDU_20220326T173142
"Bounding Box: [73.90051964, 34.24810731, 75.10727509, 35.24303017]"
Datetime: 2022-03-26 05:46:39.024000+00:00
datetime: 2022-03-26T05:46:39.024000Z
platform: Sentinel-2B
proj:epsg: 32643
instruments: ['msi']
s2:mgrs_tile: 43SDU
constellation: Sentinel 2
s2:granule_id: S2B_OPER_MSI_L2A_TL_ESRI_20220326T173143_A026386_T43SDU_N04.00
eo:cloud_cover: 0.000873


In [60]:
data_loader = create_data_loader(mini_stac_gdf, satellite_config, featurization_params['batch_size'])

In [61]:
model = RCF(featurization_params['num_features'], featurization_params['kernel_size'], len(satellite_config['bands']))

In [86]:
X_features = create_features(data_loader, featurization_params['num_features'], len(mini_stac_gdf), 
                             model, device, satellite_config['min_image_edge'])

  0%|                                                                                             | 0/2 [00:00<?, ?it/s]
0it [00:00, ?it/s]

0
1
2
3
4



 50%|██████████████████████████████████████████▌                                          | 1/2 [00:59<00:59, 59.44s/it]
0it [00:00, ?it/s]

5
6
7
8
9



100%|█████████████████████████████████████████████████████████████████████████████████████| 2/2 [01:48<00:00, 54.24s/it]


False

In [83]:
X_features.shape

(10, 500)

In [64]:
satellite_config['min_image_edge']

90

In [67]:
image.shape

torch.Size([9, 40, 40])

In [37]:
import numpy as np

In [85]:
%%time

n_points = len(mini_stac_gdf)
n_features = featurization_params['num_features']
min_image_edge = satellite_config['min_image_edge']

features_array = np.full([n_points, n_features], np.nan, dtype=float)
torch_device = torch.device('cpu')

i = -1
for images in data_loader:
    print("hello")
    for i, image in enumerate(images, start=i+1):
        print("in image")
        if image is not None:
            print(image.shape)
            if (image.shape[1] >= min_image_edge) and (
                image.shape[2] >= min_image_edge
            ):
                features_array[i] = featurize(image, model, torch_device)
                print(i, flush=True)
            else:
                print("too small")
        else:
            print("warn", flush=True)
            warnings.warn("No image found")


KeyboardInterrupt



In [ ]:
images[1].shape

In [ ]:
features_array

In [15]:
images = next(iter(data_loader))

In [16]:
from src.featurize.nn_forward_pass import featurize
import torch

In [21]:
device = torch.device('cpu')

In [20]:
featurization_params['device']

'cpu'

In [29]:
featurize(images[0], model, device)

array([[8.06232076e-03, 1.23085454e-01, 1.66685686e-01, 1.59304965e+00,
        0.00000000e+00, 3.84266339e-02, 2.21260571e+00, 3.71617228e-01,
        4.60120440e-02, 0.00000000e+00, 1.43776083e+00, 0.00000000e+00,
        5.33628762e-02, 8.07838917e-01, 0.00000000e+00, 2.83079315e-02,
        7.32741654e-02, 7.86479712e-02, 0.00000000e+00, 2.82675568e-02,
        4.74897176e-02, 6.55502779e-03, 1.04303151e-01, 1.44371269e-02,
        1.91987026e-03, 1.45500526e-01, 2.94744794e-04, 3.97824019e-01,
        8.42198208e-02, 5.50981581e-01, 8.64195764e-01, 9.57356691e-02,
        5.23914099e-01, 0.00000000e+00, 3.44401179e-03, 1.82915553e-01,
        6.63832307e-01, 2.73044389e-02, 1.64094791e-02, 3.95785226e-03,
        1.18525696e+00, 0.00000000e+00, 2.34467816e+00, 2.40163703e-04,
        0.00000000e+00, 0.00000000e+00, 8.48301798e-02, 2.32180078e-02,
        1.66538099e-04, 4.11565043e-03, 1.20734298e+00, 2.02758834e-02,
        3.90718319e-03, 1.28586544e-04, 6.39117672e-04, 1.291955